<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter6/Change-Points_Anomalies.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly detection

In [ ]:
!pip install alibi_detect

In [ ]:
from alibi_detect.datasets import fetch_kdd

intrusions = fetch_kdd()

In [ ]:
intrusions["target"].sum() / len(intrusions["target"])

In [ ]:
intrusions["feature_names"]

In [ ]:
scores = od.score(intrusions["data"][:, 0])

In [ ]:
import pandas as pd

pd.Series(intrusions["data"][:, 0]).plot();

In [ ]:
from alibi_detect.od import SpectralResidual

od = SpectralResidual(
    threshold=1.0, window_amp=20, window_local=20, n_est_points=10, n_grad_points=5
)
intrusion_outliers = od.predict(intrusions["data"][:,0])

In [ ]:
import matplotlib

ax = pd.Series(intrusions["data"][:, 0], name="data").plot(
    legend=False, figsize=(12, 6)
)
ax2 = ax.twinx()
ax = pd.Series(scores, name="scores").plot(
    ax=ax2, legend=False, color="r", marker=matplotlib.markers.CARETDOWNBASE
)
ax.figure.legend(bbox_to_anchor=(1, 1), loc="upper left");

# Change point detection

In [ ]:
!pip install ruptures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ruptures as rpt

plt.style.use("seaborn-whitegrid")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

In [ ]:
signal, bkps = rpt.pw_constant(
    n_samples=500, n_features=3, n_bkps=2, noise_std=5.0, delta=(1, 20)
)

In [ ]:
rpt.display(signal, bkps)

In [ ]:
signal.shape

In [ ]:
bkps

In [ ]:
# "l1", "rbf", "linear", "normal", "ar"
algo = rpt.Binseg(model="l1").fit(signal)
my_bkps = algo.predict(n_bkps=3)

# show results
rpt.show.display(signal, bkps, my_bkps, figsize=(10, 6))